In [1]:
using COBREXA

Let's first get a simple model to have a look at

In [2]:
import Downloads
Downloads.download(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
)

"e_coli_core.json"

Load the model

In [3]:
ecoli = load_model("e_coli_core.json");

have a look at what the model contains. First, metabolites

In [4]:
metabolites(ecoli)

72-element Vector{String}:
 "glc__D_e"
 "gln__L_c"
 "gln__L_e"
 "glu__L_c"
 "glu__L_e"
 "glx_c"
 "h2o_c"
 "h2o_e"
 "h_c"
 "h_e"
 ⋮
 "f6p_c"
 "fdp_c"
 "for_c"
 "for_e"
 "fru_e"
 "fum_c"
 "fum_e"
 "g3p_c"
 "g6p_c"

reactions

In [5]:
reactions(ecoli)

95-element Vector{String}:
 "PFK"
 "PFL"
 "PGI"
 "PGK"
 "PGL"
 "ACALD"
 "AKGt2r"
 "PGM"
 "PIt2r"
 "ALCD2x"
 ⋮
 "MALt2_2"
 "MDH"
 "ME1"
 "ME2"
 "NADH16"
 "NADTRHD"
 "NH4t"
 "O2t"
 "PDH"

we can have a look at what each reaction does

In [6]:
reaction_stoichiometry(ecoli, "PFK")

Dict{String, Float64} with 5 entries:
  "adp_c" => 1.0
  "atp_c" => -1.0
  "f6p_c" => -1.0
  "fdp_c" => 1.0
  "h_c"   => 1.0

or look at which metabolites belong to which compartments (Julia looping!)

In [7]:
[m => metabolite_compartment(ecoli, m) for m in metabolites(ecoli)]

72-element Vector{Pair{String, String}}:
 "glc__D_e" => "e"
 "gln__L_c" => "c"
 "gln__L_e" => "e"
 "glu__L_c" => "c"
 "glu__L_e" => "e"
    "glx_c" => "c"
    "h2o_c" => "c"
    "h2o_e" => "e"
      "h_c" => "c"
      "h_e" => "e"
            ⋮
    "f6p_c" => "c"
    "fdp_c" => "c"
    "for_c" => "c"
    "for_e" => "e"
    "fru_e" => "e"
    "fum_c" => "c"
    "fum_e" => "e"
    "g3p_c" => "c"
    "g6p_c" => "c"

or try to get human-readable names from the reactions

In [8]:
sort([r => reaction_name(ecoli, r) for r in reactions(ecoli)])

95-element Vector{Pair{String, String}}:
    "ACALD" => "Acetaldehyde dehydrogenase (acetylating)"
   "ACALDt" => "Acetaldehyde reversible transport"
     "ACKr" => "Acetate kinase"
   "ACONTa" => "Aconitase (half-reaction A, Citrate hydro-lyase)"
   "ACONTb" => "Aconitase (half-reaction B, Isocitrate hydro-lyase)"
    "ACt2r" => "Acetate reversible transport via proton symport"
     "ADK1" => "Adenylate kinase"
    "AKGDH" => "2-Oxogluterate dehydrogenase"
   "AKGt2r" => "2 oxoglutarate reversible transport via symport"
   "ALCD2x" => "Alcohol dehydrogenase (ethanol)"
            ⋮
 "SUCCt2_2" => "Succinate transport via proton symport (2 H)"
   "SUCCt3" => "Succinate transport out via proton antiport"
    "SUCDi" => "Succinate dehydrogenase (irreversible)"
   "SUCOAS" => "Succinyl-CoA synthetase (ADP-forming)"
     "TALA" => "Transaldolase"
     "THD2" => "NAD(P) transhydrogenase"
     "TKT1" => "Transketolase"
     "TKT2" => "Transketolase"
      "TPI" => "Triose-phosphate isomerase

this is useful to e.g. quickly look up dehumanized gene IDs

In [9]:
gs = genes(ecoli)
sort(gene_name.(Ref(ecoli), gs) .=> gs)

137-element Vector{Pair{String, String}}:
     "" => "s0001"
 "aceA" => "b4015"
 "aceB" => "b4014"
 "aceE" => "b0114"
 "aceF" => "b0115"
 "ackA" => "b2296"
 "acnA" => "b1276"
 "acnB" => "b0118"
 "adhE" => "b1241"
 "adhP" => "b1478"
        ⋮
 "talB" => "b0008"
 "tdcD" => "b3115"
 "tdcE" => "b3114"
 "tktA" => "b2935"
 "tktB" => "b2465"
 "tpiA" => "b3919"
 "ydjI" => "b1773"
 "ytjC" => "b4395"
  "zwf" => "b1852"

the model's internals are described by a bipartite graph between reactions
and metabolites, commonly stoichiometry

In [10]:
stoichiometry(ecoli)

72×95 SparseArrays.SparseMatrixCSC{Float64, Int64} with 360 stored entries:
⎡⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢄⠀⠀⠀⠀⠀⠀⠀⠈⠶⠴⡆⠀⠀⠀⠀⠀⠀⎤
⎢⡀⢐⣀⢀⡀⡒⢒⣐⠀⣂⣂⠀⣂⣂⢂⠀⢀⠀⠀⠀⠀⠀⢀⠄⠀⠀⠀⢂⠀⢂⣀⣐⡒⡀⠆⢙⣀⠀⡀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⠀⠰⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⠀⡀⠀⠀⠀⠀⠈⢑⣀⣀⠀⠀⎥
⎢⠀⠀⠃⠀⠃⠀⠀⠀⠘⠀⡇⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠀⠀⡜⠀⡄⣤⢠⠘⠙⢣⡇⠘⎥
⎢⠀⠐⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠐⠁⠉⠀⠀⠀⠀⠀⠘⠄⎥
⎢⠀⢐⠀⠂⠀⠄⠠⠠⠀⠠⠆⠀⠄⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠠⠀⠠⠀⠀⢀⠀⠀⠠⠀⠀⠁⎥
⎢⢀⠐⠀⠨⢀⠁⠈⣈⠀⢁⣁⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠄⠀⠁⢀⠀⢊⠉⠀⠀⠀⢀⠀⣀⠀⢀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡈⠀⡀⠆⠀⠆⠀⡀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠆⠀⎥
⎢⠀⠀⠂⠀⡂⠀⠀⠁⠀⠀⠀⠈⠁⠀⠀⠀⠄⠄⢁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⎥
⎢⠈⠀⠁⠀⠀⢀⡀⠀⠠⠁⠁⠀⠑⠀⠐⠲⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠂⠀⠂⠀⠀⠀⠀⠀⠀⠊⠀⠀⠀⠈⎥
⎢⠄⠠⢠⠀⠰⠀⠠⠀⠤⠦⠄⠈⠀⠀⠀⠠⠀⠁⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠤⠄⠄⠠⠀⠀⠀⠀⠀⎥
⎢⠂⠐⠀⠀⠐⡠⢐⠘⢃⠒⠂⡀⠄⠀⠀⠐⠀⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠒⠀⢀⢀⠀⠀⣀⠀⢀⎥
⎢⠈⠀⠁⠀⡀⠀⠀⠀⠈⠁⠅⠀⠁⠀⢀⠈⠄⠔⠀⠀⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠈⎥
⎢⠣⠁⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⠀⠀⠈⠀⠁⠁⠀⠈⡀⠀⠀⠀⠀⠀⠐⢣⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎣⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⡄⠀⠀⠀⠀⠂⠄⠤⠀⠀⠈⠂⠀⠀⠀⠀⠠⠀⠊⠒⣠⠀⠀⠀⠀⠀⠀⠀⠀⠀⎦

(the matrix is likely zoomed out, but otherwise it's a normal matrix)

Let's try to build a tiny custom model
ref: https://lcsb-biocore.github.io/COBREXA.jl/stable/examples/04b_standardmodel_construction/

The model type for "manually constructed" models we call a StandardModel
because it is kinda standard way to handle stuff in cobra community.
Technically we can convert the above model to StandardModel and play with it:

In [11]:
ecoli = convert(StandardModel, ecoli)
ecoli.reactions["PFK"]
ecoli.reactions["PFK"].lb = 0.5

0.5

But we want to make a completely custom model

In [12]:
m = StandardModel("MyModel")

Metabolic model of type COBREXA.StandardModel
sparse(Int64[], Int64[], Float64[], 0, 0)
Number of reactions: 0
Number of metabolites: 0


let's make some metabolites

In [13]:
a = Metabolite("a", name = "molecule A", formula = "H2O", compartment = "outside")
b = Metabolite("b") #details omitted for demonstration
c = Metabolite("c")

Metabolite.id: c
Metabolite.name: ---
Metabolite.formula: ---
Metabolite.charge: ---
Metabolite.compartment: ---
Metabolite.notes: ---
Metabolite.annotations: ---


Push the prepared metabolites into the model (the ! stands for "execute!", it
is a syntactic convention for warning that the function changes the model in
place)

In [14]:
add_metabolites!(m, [a, b, c])

let's make some reactions

In [15]:
bs = [Reaction("b$i", lb = 0.0, ub = 10.0) for i = 1:3]
bs[1].metabolites = Dict("a" => 1)
bs[2].metabolites = Dict("b" => -1)
bs[3].metabolites = Dict("c" => -1)
vs = [Reaction("v$i", lb = 0.0, ub = 10.0) for i = 1:3]
vs[1].metabolites = Dict("a" => -1, "b" => 1)
vs[2].metabolites = Dict("a" => -1, "c" => 1)
vs[3].metabolites = Dict("a" => 1, "c" => -1)

Dict{String, Int64} with 2 entries:
  "c" => -1
  "a" => 1

add the reactions to the model

In [16]:
add_reactions!(m, [bs; vs])

let's have a look at what it looks like as a matrix

In [17]:
stoichiometry(m)

3×6 SparseArrays.SparseMatrixCSC{Float64, Int64} with 9 stored entries:
 1.0    ⋅     ⋅   -1.0  -1.0   1.0
  ⋅   -1.0    ⋅    1.0    ⋅     ⋅ 
  ⋅     ⋅   -1.0    ⋅    1.0  -1.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*